In [2]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

Using TensorFlow backend.


In [51]:
# create RNN data

datafilestrain1 = os.listdir('data_complete')
datafiles = [a for a in datafilestrain1 if a.startswith('AS')]
datafiles.remove('AS14.32.csv')

################

window=1

for user in datafiles:
    for fold in [1,2,3,4]:
        train_dataset  = pd.read_csv('data_complete/fold_{}/train/{}'.format(fold, user))
        test_dataset = pd.read_csv('data_complete/fold_{}/test/'.format(fold) + user)

        test_indices = list(test_dataset['Unnamed: 0'])
        print test_indices
        train_indices = list(train_dataset['Unnamed: 0'])
        print train_indices

        df = pd.concat([train_dataset, test_dataset])
        df = df.sort_values('Unnamed: 0')

        normalized_test_df = pd.read_csv('data_normalized/fold_{}/test/{}'.format(fold, user))
        normalized_train_df = pd.read_csv('data_normalized/fold_{}/train/{}'.format(fold, user))
        vars1 = list(normalized_train_df.columns)
        #print vars1
        vars1.remove('Unnamed: 0')
        vars2 = ['predict_day', 'user', 'window', 'true_mood', 'indexje']
        cols=vars2+vars1

        dataset_df = pd.DataFrame(columns=cols)

        days = list(df['Unnamed: 0.1'])
        for i in range(len(df)):
            if i + window <= len(df) - 1:
                windowstart = days[i]
                windowend = days[i+window-1]
                predictday = days[i+window]

                avg_mood = df[i:i+window]['mood']
                avg_arousal = df[i:i+window]['arousal']
                avg_valence = df[i:i+window]['valence']
                avg_activity = df[i:i+window]['activity']

                normalized_row = normalized_test_df.loc[normalized_test_df['Unnamed: 0'] == days[i]]
                if normalized_row.empty:
                    normalized_row = normalized_train_df.loc[normalized_train_df['Unnamed: 0'] == days[i]]

                vardict = {}
                for var in vars1:
                    vardict['predict_day'] = predictday
                    vardict['mood'] = float(avg_mood)
                    vardict['arousal'] = float(avg_arousal)
                    vardict['valence'] = float(avg_valence)
                    vardict['activity'] = float(avg_activity)
                    vardict['true_mood'] = list(df['mood'])[i+window]
                    vardict['indexje'] = list(df['Unnamed: 0'])[i]
                    vardict['user'] = user[:-4]
                    vardict['window'] = '{}-{}'.format(windowstart, windowend)
                    if var not in ['mood', 'arousal', 'valence', 'activity', 'Unnamed: 0']:
                        vardict[var] = float(normalized_row[var])

                dataset_df = dataset_df.append(vardict, ignore_index=True)

        dataset_df.indexje = dataset_df.indexje.astype(int)
        dataset_df.index = dataset_df['indexje']

        test_data = dataset_df.loc[[a-window for a in test_indices]]
        train_data = dataset_df.loc[[a-window for a in train_indices]]

        train_data = train_data[pd.notnull(train_data['predict_day'])]
        test_data = test_data[pd.notnull(test_data['predict_day'])]
        train_data = train_data.drop('indexje', axis=1)
        test_data = test_data.drop('indexje', axis=1)

        train_data.to_csv('~/DataMiningTechniques2018/Ass1/data_rnn/fold_{}/train/{}'.format(fold, user), index=False)
        test_data.to_csv('~/DataMiningTechniques2018/Ass1/data_rnn/fold_{}/test/{}'.format(fold, user), index=False)
        
        print user, fold


def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back):
        a = dataset[i:(i+look_back)]
        dataX.append(a)
        dataY.append(dataset[i + look_back])
    return dataX, dataY

look_back = 1

def reshape_etc(data):
    size = len(data)
    data = np.array(data).reshape(size, 1)
    return data

trainX, trainY = create_dataset(np.array(train['mood']), look_back)
testX, testY = create_dataset(np.array(test['mood']), look_back)

valence_train_X, valence_train_Y = create_dataset(np.array(train['valence']), look_back)
arousal_train_X, arousal_train_Y = create_dataset(np.array(train['arousal']), look_back)
activity_train_X, activity_train_Y = create_dataset(np.array(train['activity']), look_back)

valence_test_X, valence_test_Y = create_dataset(np.array(test['valence']), look_back)
arousal_test_X, arousal_test_Y = create_dataset(np.array(test['arousal']), look_back)
activity_test_X, activity_test_Y = create_dataset(np.array(test['activity']), look_back)

trainX, trainY = reshape_etc(trainX), reshape_etc(trainY)
valence_train_X = reshape_etc(valence_train_X)
arousal_train_X = reshape_etc(arousal_train_X)
activity_train_X = reshape_etc(activity_train_X)
valence_test_X = reshape_etc(valence_test_X)
arousal_test_X = reshape_etc(arousal_test_X)
activity_test_X = reshape_etc(activity_test_X)

trainX = np.column_stack((trainX, valence_train_X, arousal_train_X, activity_train_X))
testX = np.column_stack((testX, valence_test_X, arousal_test_X, activity_test_X))     
 
trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))


[8, 9, 10, 11, 12, 13, 14, 15]
[0, 1, 2, 3, 4, 5, 6, 7, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39]


IOError: [Errno 2] No such file or directory: '/home/s1924192/DataMiningTechniques2018/Ass1/data_rnn/fold_1/train/AS14.12.csv'

In [ ]:
trainY

In [ ]:
import os

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "4"

model = Sequential()
model.add(LSTM(4, input_shape=(1, look_back)))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(trainX, trainY, epochs=100, batch_size=1, verbose=2)

In [10]:
import numpy as np
import pandas as pd
import os

fold = 1

trainfiles = os.listdir('data_complete/fold_{}/train'.format(1))
testfiles = os.listdir('data_complete/fold_{}/test'.format(1))

try:
    trainfiles.remove('AS14.32.csv')
    testfiles.remove('AS14.32.csv')
except:
    pass
    
for user in [trainfiles[0]]:
    for fold in [[1, 2, 3, 4][0]]:
        for window in [1]:
            train_dataset  = pd.read_csv('data_complete/fold_{}/train/'.format(fold) + user)
            test_dataset = pd.read_csv('data_complete/fold_{}/test/'.format(fold) + user)

            test_indices = list(test_dataset['Unnamed: 0'])
            print test_indices
            train_indices = list(train_dataset['Unnamed: 0'])
            print train_indices

            df = pd.concat([train_dataset, test_dataset])
            df = df.sort_values('Unnamed: 0')

            dataset_df = pd.DataFrame(columns=['predict_day', 'user', 'window', 'mood', 'arousal', \
                                                   'valence', 'activity', 'true_mood', 'indexje'])

            #print df
            days = list(df['Unnamed: 0.1'])
            for i in range(len(df)):
                if i + window <= len(df) - 1:
                    windowstart = days[i]
                    windowend = days[i+window-1]
                    predictday = days[i+window]

                    avg_mood = df[i:i+window]['mood'].mean()
                    avg_arousal = df[i:i+window]['arousal'].mean()
                    avg_valence = df[i:i+window]['valence'].mean()
                    avg_activity = df[i:i+window]['activity'].mean()

                    dataset_df = dataset_df.append({'predict_day': predictday, 'user': user[:-4], 'window': '{}-{}'.format(windowstart, \
                                                     windowend), 'mood': avg_mood, 'arousal': avg_arousal, \
                                                     'valence': avg_valence, 'activity': avg_activity, \
                                                     'true_mood': list(df['mood'])[i+window], 'indexje': list(df['Unnamed: 0'])[i]}, ignore_index=True)

            dataset_df.indexje = dataset_df.indexje.astype(int)
            dataset_df.index = dataset_df['indexje']

            test_data = dataset_df.loc[[a-window for a in test_indices]]
            train_data = dataset_df.loc[[a-window for a in train_indices]]

            train_data = train_data[pd.notnull(train_data['predict_day'])]
            test_data = test_data[pd.notnull(test_data['predict_day'])]
            train_data = train_data.drop('indexje', axis=1)
            test_data = test_data.drop('indexje', axis=1)
            #train_data.to_csv('interval_datasets/window_{}/fold_{}/train/{}'.format(window, fold, user), index=False)
            #test_data.to_csv('interval_datasets/window_{}/fold_{}/test/{}'.format(window, fold, user), index=False)
            print train_data
            print test_data

[8, 9, 10, 11, 12, 13, 14, 15]
[0, 1, 2, 3, 4, 5, 6, 7, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39]
        predict_day     user                 window  mood  arousal   valence  \
indexje                                                                        
0        2014-03-28  AS14.12  2014-03-27-2014-03-27  7.00    -0.50  1.000000   
1        2014-03-29  AS14.12  2014-03-28-2014-03-28  6.75     0.00  0.750000   
2        2014-03-30  AS14.12  2014-03-29-2014-03-29  6.75     1.00  1.000000   
3        2014-03-31  AS14.12  2014-03-30-2014-03-30  5.80    -0.20  0.250000   
4        2014-04-01  AS14.12  2014-03-31-2014-03-31  6.40     0.40  0.400000   
5        2014-04-02  AS14.12  2014-04-01-2014-04-01  6.40     0.40  0.750000   
6        2014-04-03  AS14.12  2014-04-02-2014-04-02  6.00    -0.40  0.333333   
15       2014-04-12  AS14.12  2014-04-11-2014-04-11  6.60    -0.50  1.000000   
16       2014-04-13  AS14.12  2014-04-12-2014-04